# Sapflow - drought analysis

## Aim of this notebook is to look at when is tree doing its best, and during those times what is soil moist, precip, GWS

In [ ]:
### 

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2013,2016)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath = os.path.join('../')
#print('datapath is set to %s'%datapath)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
import matplotlib.dates as mdates
#from colorspacious import cspace_converter
import scipy.stats as stats
import seaborn as sns
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile
from Ags_model import runAgs, calc_LE

### import data

#### sapflow import

In [ ]:
Sapf_data = pd.read_csv('Sapflow.csv')
Sapf_data.loc[:,'TIMESTAMP'] = pd.to_datetime(Sapf_data.loc[:,'TIMESTAMP'])
Sapf_data.index=Sapf_data['TIMESTAMP']
Sapf_data.drop(columns='TIMESTAMP',inplace=True)
Sapf_data

#### soil moisture import

In [ ]:
SM = pd.read_csv('Soil_moisture.csv')
SM.loc[:,'datetime'] = pd.to_datetime(SM.loc[:,'datetime'])
SM.index=SM['datetime']
SM.drop(columns='datetime',inplace=True)
SM

#### Groundwater import

In [ ]:
GWS = pd.read_csv('GWS_daily.csv')
GWS.loc[:,'datetime'] = pd.to_datetime(GWS.loc[:,'datetime'])
GWS.index=GWS['datetime']
GWS.drop(columns='datetime',inplace=True)
GWS

#### dendrometer import

#### LE, GPP import, calc WUE

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    progress.append('dataloaded')

In [ ]:
#from FilterData import Filter_wrap
#CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')

In [ ]:
#alternate filtering
from FilterData import Filter_GPP_LE_NEE_VPD
df_Comb_alt_filter = Filter_GPP_LE_NEE_VPD(df_Comb,fqc=None)
df_Comb_alt_filter.columns

In [ ]:
df_Comb=df_Comb_alt_filter.loc[:,['GPP_f','LE','LE_f','H','H_f','VPD']]
df_Comb['WUE']=df_Comb['GPP_f']/df_Comb['LE']

In [ ]:
df_Comb

In [ ]:
#df_Comb
st='2014-11-20 23:00'
ed='2014-11-21 02:00'
df_Comb.loc[st:ed,'WUE'].plot()
df_Comb.loc[st:ed,'LE'].plot()
df_Comb.loc[st:ed,'GPP_f'].plot()
print('GPP: ',df_Comb.loc[st:ed,'GPP_f'])
print('LE: ',df_Comb.loc[st:ed,'LE'])


In [ ]:
#import seaborn as sns

#ax = sns.histplot(data= df_Comb['WUE'])
#ax = df_Comb['WUE'].hist(bins=100)
#ax.set_xlim(-500,500)

In [ ]:
ax = df_Comb['WUE'].plot()
ax.set_ylim(-50,50)

### end of import data

### WUE calculation

In [ ]:
# 1: select days for which we have all LE and GPP for daytime (8am to 8pm)
count_LE = df_Comb.loc[:,'LE'].between_time('09:00','18:00').resample('D').count()
amount_LE=count_LE.loc[count_LE==19].count()
print('amount of days that have 19 entries for 09:00-18:00 - ',amount_LE)
any_count_LE = df_Comb.loc[:,'LE'].resample('D').count()
any_amount_LE=any_count_LE.loc[any_count_LE!=0].count()
print('amount of days that have any amount of entries for 00:00-23:30 - ',any_amount_LE)

In [ ]:
# 1: same for LE_f(gapfilled)
count_LE_f = df_Comb.loc[:,'LE_f'].between_time('09:00','18:00').resample('D').count()
amount_LE_f=count_LE_f.loc[count_LE_f==19].count()
print('amount of days that have 19 entries for 09:00-18:00 - ',amount_LE)
any_count_LE_f = df_Comb.loc[:,'LE_f'].resample('D').count()
any_amount_LE_f=any_count_LE_f.loc[any_count_LE_f!=0].count()
print('amount of days that have any amount of entries for 00:00-23:30 - ',any_amount_LE)

In [ ]:
# 1: select days for which we have all LE and GPP for daytime (8am to 8pm)
count_GPP = df_Comb.loc[:,'GPP_f'].between_time('09:00','18:00').resample('D').count()
amount_GPP=count_GPP.loc[count_GPP==19].count()
print('amount of days that have 19 entries for 09:00-18:00 - ',amount_GPP)
any_count_GPP = df_Comb.loc[:,'GPP_f'].resample('D').count()
any_amount_GPP=any_count_GPP.loc[any_count_GPP!=0].count()
print('amount of days that have any amount of entries for 00:00-23:30 - ',any_amount_GPP)

In [ ]:
df_Comb_GPPLE=df_Comb.loc[:,['GPP_f','LE']].where((count_LE_f==19)&(count_GPP==19))
df_Comb_GPPLE['GPP_sum']= df_Comb_GPPLE['GPP_f'].resample('D').sum()
df_Comb_GPPLE['GPP_sum'].where(df_Comb_GPPLE['GPP_sum']!=0.0,np.nan,inplace=True)
df_Comb_GPPLE['LE_sum']= df_Comb_GPPLE['LE'].resample('D').sum()
df_Comb_GPPLE.loc[:,'LE_sum'].where(df_Comb_GPPLE['LE_sum']!=0.0,np.nan,inplace=True)

df_Comb_GPPLE.dropna(how='any',inplace=True)
df_Comb_GPPLE.loc[:,'GPP_sum'].where(df_Comb_GPPLE['GPP_sum']==0,np.NaN)
df_Comb_GPPLE['WUE']= df_Comb_GPPLE['GPP_sum']/df_Comb_GPPLE['LE_sum']

df_Comb_GPPLE.loc[:,'WUE'].plot()

In [ ]:
(df_Comb_GPPLE['GPP_sum']==0)&(df_Comb_GPPLE['LE_sum']==0)

In [ ]:
count_LE.plot()

In [ ]:
#store data for heatmap
#fun little sideproject [WIP]
#df_Comb['entries_LE'] = df_Comb.loc[:,'LE'].between_time('09:00','18:00').resample('D').count()
#df_Comb['entries_GPP'] = df_Comb.loc[:,'LE'].between_time('09:00','18:00').resample('D').count()
df_Comb['flag_amount_LE'] = 0
df_Comb['flag_amount_GPP'] = 0
df_Comb['flag_amount_both'] = 0
df_Comb['flag_amount_LE'] = df_Comb.loc[:,'LE'].between_time('09:00','18:00').resample('D').count().where(count_LE==19,1)
df_Comb['flag_amount_GPP'] = df_Comb.loc[:,'GPP'].between_time('09:00','18:00').resample('D').count().where(count_GPP==19,1)
df_Comb['flag_amount_both'] = w

In [ ]:
# 2: integrate over the day GPP and LE
# 3: wue = GP/ET


In [ ]:
# lets make a heatmap
sns.heatmap()

### sapflow: top5%

In [ ]:
Sapf_data['Jt_mean'].plot.hist()

In [ ]:
sap_95quan=Sapf_data['Jt_mean'].quantile(q=0.95) #top 5% quantile is above....
print(sap_95quan)

In [ ]:
Sapf_top5=Sapf_data.loc[:,['Jt_mean']].loc[Sapf_data['Jt_mean']>=sap_95quan]
Sapf_top5

In [ ]:
Sapf_top5.plot()

In [ ]:
Sapf_top5.boxplot()